In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [10]:
llm=OpenAI(model='text-davinci-003',temperature=0)

Text of the context was copied from: https://www.newser.com/story/337752/4-people-attacked-by-dolphins-on-japan-beach.html

## I. Simple Prompt Template with context

In [11]:
context="""Over the course of one day on a central Japan beach, four people were attacked by dolphins. Early Sunday, a man in his 60s was attacked a few yards off the popular Suishohama Beach in the town of Mihama, Fukui prefecture; he was left with broken ribs and bites to his hands, the BBC reports. Also that morning, a man in his 40s was attacked at the same beach and suffered bites to his arm, MSN reports. Later that day, two more people were hurt in dolphin attacks. That brings the total number of dolphin attacks for the prefecture this year to six; similar attacks were also recorded last year. 
Such attacks are unusual, but experts have warned that bottlenose dolphins are stressed out when they must swim near humans, and that having to do so disrupts their behavioral routines. "Dolphins are wild animals and may see humans as a potential threat to their territory or prey. In some cases, dolphins may also attack humans as a form of play or amusement, which can be dangerous due to their size and strength," American Oceans explains. Signs at Suishohama Beach now warn swimmers not to touch or approach dolphins. "If you see them, don't go into the water," an official warns."""

In [12]:
prompt=PromptTemplate(
            template="Use the context: "+context+ " to answer the questions. If the question cannot be answered using the context provided, respond with I don't know\
            Question:{question}\
            Answer:", 
            input_variables=['question'])

In [13]:
llmchain=LLMChain(llm=llm, prompt=prompt)

In [14]:
response1=llmchain.run(question='Where was the attack by the dolphins happen?')

In [15]:
print(response1)

The attack by the dolphins happened at Suishohama Beach in the town of Mihama, Fukui prefecture.


In [16]:
response2=llmchain.run(question='Why did the dolphins attach the people?')

In [17]:
response2

'The dolphins may have seen the people as a potential threat to their territory or prey, or they may have attacked them as a form of play or amusement.'

In [18]:
response3=llmchain.run('What should we do if we see dolphins?')

In [19]:
print(response3)

If you see them, don't go into the water.


In [20]:
response4=llmchain.run('What should we do if we see a shark?')

In [21]:
print(response4)

I don't know.


## II. Few Shot Prompting

In [22]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [23]:
examples=[
    {'query':'lion','response':'savannah'},
    {'query':'polar bear','response':'artic ice'},
    {'query':'elephants','response':'African grasslands'}
]

In [24]:
example_template='''
query:{query}
response:{response}'''

In [25]:
prefix='Answer the question by providing the natural habitat of {query}'

In [26]:
suffix='''
query={query}
response='''

In [31]:
fspt_habitat=FewShotPromptTemplate(
    prefix=prefix,
    example_prompt=PromptTemplate(template=example_template,input_variables=['query','response']),
    examples=examples,
    suffix=suffix,
    input_variables=['query']
)

In [32]:
llmchain=LLMChain(prompt=fspt_habitat,llm=llm)

In [33]:
response_fspt=llmchain.run('penguin')

In [34]:
response_fspt

'Antarctic and sub-Antarctic regions.'

In [35]:
response_fspt1=llmchain.run('men')

In [36]:
response_fspt1

'various habitats around the world'

In [37]:
response_fspt2=llmchain.run('women')

In [38]:
response_fspt2

'everywhere'

In [39]:
response_fspt3=llmchain.run('eagles')

In [40]:
response_fspt3

'mountain ranges, forests, and coastal areas.'

In [41]:
response_fspt4=llmchain.run('blue whales')

In [42]:
response_fspt4

'oceans and seas'

In [43]:
response_fspt5=llmchain.run('pigeons')

In [44]:
response_fspt5

'urban areas, woodlands, and grasslands.'

## Prompting exercises with sarcasm

In [50]:
sarcasm_examples=[
    {'question':'Are you going to sleep?','response':'No. I am practicing how to die.'},
    {'question':'Are you awake already?','response':'Of course not. I am sleepwalking'},
    {'question':'Is this the elevator to go up?','response':'I think this is the elevator that goes sideways'},
    {'question':'Does obesity run in our family?','response':'No one runs in your family'},
    {'question':'How much do you earn?','response':'Enough to call a cab to take me away from that stupid question'}
]

In [53]:
sarcasm_template='question:{question},response:{response}'

In [54]:
sarcasm_prompt=PromptTemplate(template=sarcasm_template, input_variables=['question','response'])

In [55]:
prefix='You are an AI agent that gives sarcastic answers to questions. Exercepts from answers you have given before are below'

In [56]:
suffix='''
question:{question}
response:'''

In [58]:
fspt_sarcasm=FewShotPromptTemplate(examples=sarcasm_examples,prefix=prefix,example_prompt=sarcasm_prompt,suffix=suffix,input_variables=['question'])

In [59]:
llmchain_sarcasm=LLMChain(prompt=fspt_sarcasm,llm=llm)

In [60]:
response1=llmchain_sarcasm.run('Why have you put on so much weight?')

In [61]:
print(response1)

Because I'm trying to make myself look more like you.


In [62]:
sarc_response1=llmchain_sarcasm.run("Why are you always so busy?")
print(sarc_response1)

Because I'm too important to be idle.


In [64]:
sarc_response2=llmchain_sarcasm.run("Why is the sky blue?")
print(sarc_response2)

Because it's jealous of how much everyone loves the ocean.


#### This was an interesting answer since the query was slightly out of distribution since the examples were all personal questions but this was a more scientific one 

## Using LengthBasedExampleSelector to avoid crossing tokenizer lengths

In [65]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [72]:
examples_physics=[
    {'question':'What is the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?', 
     'answer':'MOND is a theory that eliminates the observed missing baryonic mass in galaxy clusters by imposing a new mathematical formulation of gravity that does not require the existence of dark matter.'},
    
    {'question':'What is dynamic scaling in self-similar systems?',
     'answer':'Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.'},
     
    {'question':'What is the origin and significance of the triskeles symbol?',
     'answer':'The triskeles symbol was reconstructed as a feminine divine triad by the rulers of Syracuse, and later adopted as an emblem. Its usage may also be related to the Greek name of Sicily, Trinacria, which means "having three headlands." The head of Medusa at the center of the Sicilian triskeles represents the three headlands.'} ,
    
    {'question':'What is the Roche limit?',
     'answer':"The Roche limit is the distance at which tidal effects would cause an object to break apart due to differential force from the planet overcoming the attraction of the parts of the object for one another, which depends on the object's density and composition, as well as the mass and size of the planet."} ,
    
    {'question':"What is Martin Heidegger's view on the relationship between time and human existence?",
     'answer':"Martin Heidegger believes that humans do not exist inside time, but that they are time. The relationship to the past is a present awareness of having been, and the relationship to the future involves anticipating a potential possibility, task, or engagement."} ,
    
    {'question':'What is the decay energy for the free neutron decay process?',
     'answer':'0.782343 MeV'} ,
    
    {'question':'What is the evidence for the existence of a supermassive black hole at the center of the Milky Way galaxy?',
     'answer':"The star S2 follows an elliptical orbit with a period of 15.2 years and a pericenter of 17 light-hours from the center of the central object. From the motion of star S2, the object's mass can be estimated as 4.0 million Mâ˜‰, or about 7.96Ã—1036 kg. The radius of the central object must be less than 17 light-hours, because otherwise S2 would collide with it. Observations of the star S14 indicate that the radius is no more than 6.25 light-hours, about the diameter of Uranus' orbit. No known astronomical object other than a black hole can contain 4.0 million Mâ˜‰ in this volume of space."} ,
    
    {'question':'What is the Carnot engine?',
     'answer':'The Carnot engine is an ideal heat engine that operates in the limiting mode of extreme slowness known as quasi-static. It represents the theoretical maximum efficiency of a heat engine operating between any two given thermal or heat reservoirs at different temperatures.'} ,
    
    {'question':'What is the second law of thermodynamics?',
     'answer':'The second law of thermodynamics is a physical law based on universal experience concerning heat and energy interconversions. It establishes the concept of entropy as a physical property of a thermodynamic system and can be used to predict whether processes are forbidden despite obeying the requirement of conservation of energy as expressed in the first law of thermodynamics.'} ,
    
    {'question':'Who shared the other half of the Nobel Prize with Yoichiro Nambu for discovering the origin of the explicit breaking of CP symmetry in the weak interactions?',
     'answer':'Makoto Kobayashi and Toshihide Maskawa'} ,
    
    {'question':'What is the Peierls bracket in canonical quantization?',
     'answer':'The Peierls bracket is a Poisson bracket derived from the action in the canonical quantization method that converts the quotient algebra into a Poisson algebra.'} ,
    
    {'question':'What is a regular polytope?',
     'answer':'A regular polytope is a geometric shape whose symmetry group is transitive on its flags.'} ,
    
    
]

In [68]:
prompt=PromptTemplate(
template='User: {question}\nAI:{answer}',
input_variables=['question','answer']
)

In [84]:
prefix='You are an AI assistant specialized in the field of Physics. Answer the {question} of the user. If you are not confident of the answer, say that you don"t know the answer. You have previously provided answers to sample questions as per the examples below'

In [85]:
suffix='Question:{question},\nAnswer:'

In [86]:
fspt_physics=FewShotPromptTemplate(
    prefix=prefix,
    example_selector=LengthBasedExampleSelector(example_prompt=prompt, examples=examples_physics,max_length=100),
    example_prompt=prompt,
    suffix=suffix,
    input_variables=['question']
)
    

In [87]:
physics_chain=LLMChain(llm=llm, prompt=fspt_physics)

In [88]:
physics_chain.run("What does Earnshaw's theorem state?")

" Earnshaw's theorem states that it is impossible to maintain a stable equilibrium configuration of forces in a system of point particles with only electrostatic interactions."

In [89]:
physics_chain.run("What happens to an initially inhomogeneous physical system that is isolated by a thermodynamic operation?")

'When an initially inhomogeneous physical system is isolated by a thermodynamic operation, it will tend to move towards a state of equilibrium, where the temperature and pressure are uniform throughout the system. This process is known as thermalization.'

##### Using scientific * humor examples

In [93]:
science_humor=[
    {
        "question": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "question": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "question": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "question": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "question": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "question": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "question": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "question": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "question": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "question": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "question": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "question": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "question": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [94]:
science_humor_prefix='You are an AI assistant to provides humorous answers with a scientific basis. You have answered previous questions as below:'

In [97]:
fspt_humor=FewShotPromptTemplate(
example_selector=LengthBasedExampleSelector(examples=science_humor,max_length=100,example_prompt=prompt),
prefix=prefix,
suffix=suffix,
example_prompt=prompt,
input_variables=['question'])

In [102]:
llm_scientifichumor=LLMChain(prompt=fspt_humor,llm=OpenAI(model='text-davinci-003',temperature=0.99))

In [103]:
llm_scientifichumor.run('Who invented the telephone?')

"Alexander Graham Bell, the original 'ringmaster'."

In [101]:
llm_scientifichumor1=LLMChain(prompt=fspt_humor,llm=OpenAI(model='text-davinci-003',temperature=0))
llm_scientifichumor1.run('Who invented the telephone?')

"Alexander Graham Bell, the original 'ringmaster'."

#### Changing the temperature where the prefix specifies some dimension of creativity doesn't impact the results